# Data cleanup and compression using parquet

 - Here we can specify the name of a text or csv file, read it, and compress it to smaller files.
 - We can export a file as parquet, which is much easier and faster to read/write in Pandas than CSV.
 - CSV is a thing of the past, and is basically only useful for portability.
 - We can also convert the integer columns to int32, float64 columns to float32, and save it for smaller data size.

Using this notebook's functionality, 300 MB CSV files can be saved as 30 MB parquet files. This way, there will never
be any need for Git LFS. We can transport the data more easily.

***NOTE:*** To use this notebook, apart from pandas, you also need to install `fastparquet` and `pyarrow` using pip or conda.

In [1]:
import pandas as pd
from IPython.display import display

# Path to the txt or csv file holding the original data, WITHOUT THE .txt or .csv EXTENSION
filepath_without_extension = "../data/2D_data_2022_11_21_clipped"

In [2]:
df = pd.read_csv(filepath_without_extension+".csv", header=0)
display(df.head())
print("Columns ...")
print(df.columns)

,direction,controller,condition,trial,emg_1,emg_2,emg_3,emg_4,velocity_norm,velocity_x,velocity_y,velocity_z,force_norm,force_x,force_y,force_z,power_total,power_x,power_y,power_z
0,1,1,1,1,0.183173,0.101127,0.467872,0.162736,0.000512,0.000074,0.0,-0.000507,2.114447,1.79587,-0.178618,-1.10174,0.001083,0.000133,0.0,0.000558
1,1,1,1,1,0.185290,0.101045,0.465411,0.162280,0.000508,-0.000030,0.0,0.000507,2.129964,1.80496,-0.186231,-1.11543,0.001081,0.000054,0.0,0.000565
2,1,1,1,1,0.188333,0.101184,0.461660,0.162284,0.000507,-0.000015,0.0,-0.000507,2.147686,1.82161,-0.178118,-1.12364,0.001089,0.000026,0.0,0.000569
3,1,1,1,1,0.191822,0.101790,0.456888,0.161915,0.000507,0.000015,0.0,0.000507,2.148192,1.82564,-0.168580,-1.11953,0.001089,0.000027,0.0,0.000567
4,1,1,1,1,0.195391,0.102374,0.453442,0.161172,0.000507,-0.000015,0.0,-0.000507,2.166847,1.84514,-0.151396,-1.12595,0.001098,0.000028,0.0,0.000570


Columns ...
Index(['direction', 'controller', 'condition', 'trial', 'emg_1', 'emg_2',
       'emg_3', 'emg_4', 'velocity_norm', 'velocity_x', 'velocity_y',
       'velocity_z', 'force_norm', 'force_x', 'force_y', 'force_z',
       'power_total', 'power_x', 'power_y', 'power_z'],
      dtype='object')


In [3]:
# Columns that hold integer or categorical data
int_cols = ['direction', 'controller', 'condition', 'trial']

In [4]:
# Look at the data types of all the columns in the data frame
df.dtypes

direction          int64
controller         int64
condition          int64
trial              int64
emg_1            float64
emg_2            float64
emg_3            float64
emg_4            float64
velocity_norm    float64
velocity_x       float64
velocity_y       float64
velocity_z       float64
force_norm       float64
force_x          float64
force_y          float64
force_z          float64
power_total      float64
power_x          float64
power_y          float64
power_z          float64
dtype: object

In [5]:
### Convert integer columns to int32 and all float64 columns to float32 to save space.

df[int_cols] = df[int_cols].astype(int)

# Select columns with 'float64' dtype  
float64_cols = list(df.select_dtypes(include='float64'))

# The same code again calling the columns
df[float64_cols] = df[float64_cols].astype('float32')

df.dtypes

direction          int32
controller         int32
condition          int32
trial              int32
emg_1            float32
emg_2            float32
emg_3            float32
emg_4            float32
velocity_norm    float32
velocity_x       float32
velocity_y       float32
velocity_z       float32
force_norm       float32
force_x          float32
force_y          float32
force_z          float32
power_total      float32
power_x          float32
power_y          float32
power_z          float32
dtype: object

In [6]:
# Save the file in the same place with the same name but as a parquet file
df.to_parquet(filepath_without_extension+".parquet", compression='gzip')

In [ ]:
# Save the new lighter dataframe as a csv file, but with int32 and float32 columns, which is lighter.
df.to_csv(filepath_without_extension+"_light.csv", index=False, header=True, float_format='%.6f')

In [7]:
# test to see if we got it right.
df2 = pd.read_parquet(filepath_without_extension+".parquet")

display(df2.head())
display(df2.dtypes)

,direction,controller,condition,trial,emg_1,emg_2,emg_3,emg_4,velocity_norm,velocity_x,velocity_y,velocity_z,force_norm,force_x,force_y,force_z,power_total,power_x,power_y,power_z
0,1,1,1,1,0.183173,0.101127,0.467872,0.162736,0.000512,0.000074,0.0,-0.000507,2.114447,1.79587,-0.178618,-1.10174,0.001083,0.000133,0.0,0.000558
1,1,1,1,1,0.185290,0.101045,0.465411,0.162280,0.000508,-0.000030,0.0,0.000507,2.129964,1.80496,-0.186231,-1.11543,0.001081,0.000054,0.0,0.000565
2,1,1,1,1,0.188333,0.101184,0.461660,0.162284,0.000507,-0.000015,0.0,-0.000507,2.147686,1.82161,-0.178118,-1.12364,0.001089,0.000026,0.0,0.000569
3,1,1,1,1,0.191822,0.101790,0.456888,0.161915,0.000507,0.000015,0.0,0.000507,2.148192,1.82564,-0.168580,-1.11953,0.001089,0.000027,0.0,0.000567
4,1,1,1,1,0.195391,0.102374,0.453442,0.161172,0.000507,-0.000015,0.0,-0.000507,2.166847,1.84514,-0.151396,-1.12595,0.001098,0.000028,0.0,0.000570


direction          int32
controller         int32
condition          int32
trial              int32
emg_1            float32
emg_2            float32
emg_3            float32
emg_4            float32
velocity_norm    float32
velocity_x       float32
velocity_y       float32
velocity_z       float32
force_norm       float32
force_x          float32
force_y          float32
force_z          float32
power_total      float32
power_x          float32
power_y          float32
power_z          float32
dtype: object